# Electricity predicteur
LSTM based model for Electricity consommation prediction

In [ ]:
# Loss
criterion = torch.nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr = lr)

# Part 1 of training
print('-- Training the model --')
train_lstm(model,
    train_dataset,
    test_dataset,
    criterion,
    optimizer,
    nepochs,
    path_logs = path_logs,
    path_model = path_model,
    batch_size = batch_size,
    device = device)

torch.save(model.state_dict(), path_model)

In [ ]:
####################
# Training a model #
####################


# Importing a LstmNet
model = LSTMNet(input_dim,
	hidden_size = hidden_dim,
	num_layers = num_layers,
	out_size = input_dim)

# Load weights if specified
if model_path != '':
    model.load_state_dict(torch.load(model_path, weights_only=True))

# Changing to appropriate device
model.to(device)

print(model)
print("Number of Parameters :", sum(p.numel() for p in model.parameters()))

In [ ]:
print('Dataset path :', dataset_path)
print('batch_size :', batch_size)

# Import the dataset
df = pd.read_csv(dataset_path)
df = df[[col]]
print(df)

# Normalizing data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_dataset = scaler.fit_transform(df)


# Separating the train and test sets
train_dataset, test_dataset = train_test_split(
    scaled_dataset,
    test_size=0.15,
    shuffle=False
)

# Get training and test size
train_size, input_dim = train_dataset.shape
test_size, _ = test_dataset.shape

print('train_size :', train_size)
print('test_size :', test_size)
print('n_features :', input_dim)

In [ ]:
# Changing to mps if available
device = torch.device("mps" if torch.mps.is_available() else "cpu")
print('Device :', device)

# Create the directory containing the model, the logs, etc.
dir_name = strftime("%Y-%m-%d_%H-%M-%S", gmtime())
out_dir = os.path.join(save_dir, dir_name)
os.makedirs(out_dir)

path_model = os.path.join(out_dir, "model.pth")
path_model_classif = os.path.join(out_dir, "model_classif.pth")
path_config = os.path.join(out_dir, "config.json")
path_logs = os.path.join(out_dir, "logs.json")

with open(path_config, 'w') as f:
    config = {
        "dataset_path": dataset_path,
        "lr": lr,
        "batch_size": batch_size,
        "nepochs": nepochs,
        "hidden_dim": hidden_dim
    }
    json.dump(config, f)
    f.write('\n')

In [ ]:
# Hyper parameters
dataset_path = "dataset.csv"
model_path = ''
lr = 1e-3
batch_size = 8
save_dir = "outputs"
nepochs = 10
num_layers = 2
hidden_dim = 64
label_smoothing = 0.0

col = 'AEP_MW'

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from time import time, strftime, gmtime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from LstmNet import *
from train_model import *